In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

In [4]:
hurdat = pd.read_csv('hurdat_cleaned.csv')
hurdat

,Unnamed: 0,date,time,record_id,status,latitude,longitude,max_wind,min_pressure,ne34ktr,...,se50ktr,sw50ktr,nw50ktr,ne64ktr,se64ktr,sw64ktr,nw64ktr,rad_max_wind,name,year
0,1,1851-06-25,0,NaN,HU,28.0N,94.8W,80.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,UNNAMED,1851
1,2,1851-06-25,600,NaN,HU,28.0N,95.4W,80.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,UNNAMED,1851
2,3,1851-06-25,1200,NaN,HU,28.0N,96.0W,80.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,UNNAMED,1851
3,4,1851-06-25,1800,NaN,HU,28.1N,96.5W,80.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,UNNAMED,1851
4,5,1851-06-25,2100,L,HU,28.2N,96.8W,80.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,UNNAMED,1851
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53971,55923,2022-11-10,1900,L,TS,29.2N,83.0W,40.0,989.0,200.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,70.0,NICOLE,2022
53972,55924,2022-11-11,0,L,TS,30.1N,84.0W,35.0,992.0,200.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,70.0,NICOLE,2022
53973,55925,2022-11-11,600,NaN,TD,31.2N,84.6W,30.0,996.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,90.0,NICOLE,2022
53974,55926,2022-11-11,1200,NaN,TD,33.2N,84.6W,25.0,999.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,250.0,NICOLE,2022


In [13]:
hurdat['status'].value_counts()

status
TS    19823
HU    15388
TD     9798
EX     6018
LO     1531
SS      699
SD      323
DB      258
WV      138
Name: count, dtype: int64

In [42]:
df2 = hurdat.query('status in ["HU","TS"] & year >= 1950')[['year','name','status']].drop_duplicates()
df2.head()

,year,name,status
21879,1950,ABLE,TS
21890,1950,ABLE,HU
21934,1950,BAKER,TS
21940,1950,BAKER,HU
21990,1950,CHARLIE,TS


In [43]:
status_order = ['TS','HU']
df2['status'] = pd.Categorical(df2['status'], categories=status_order, ordered=True)
max_status_df = df2.loc[df2.groupby(['year', 'name'])['status'].idxmax()]
stormcount_long = max_status_df.groupby(['year','status']).nunique('name').reset_index().rename(columns={'name':'count'})
stormcount_long.head()

/var/folders/rk/9lppjzsx39ncspl8m_4lnnn40000gn/T/ipykernel_56304/3158534243.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  stormcount_long = max_status_df.groupby(['year','status']).nunique('name').reset_index().rename(columns={'name':'count'})


,year,status,count
0,1950,TS,3
1,1950,HU,11
2,1951,TS,3
3,1951,HU,8
4,1952,TS,2


In [45]:
stormcount = stormcount_long.pivot(index = 'year', columns = 'status', values = 'count').reset_index().fillna(0)
stormcount['TOTAL'] = stormcount['TS'] + stormcount['HU']
stormcount

status,year,TS,HU,TOTAL
0,1950,3,11,14
1,1951,3,8,11
2,1952,2,5,7
3,1953,2,7,9
4,1954,3,6,9
...,...,...,...,...
68,2018,7,8,15
69,2019,10,6,16
70,2020,15,14,29
71,2021,13,7,20


In [46]:
stormcount.to_csv('nstorms_byyear.csv')

In [40]:
hurdat.query('year==2022')['name'].value_counts()

name
DANIELLE    62
FIONA       61
BONNIE      55
EARL        52
IAN         40
GASTON      34
NICOLE      26
LISA        24
JULIA       21
MARTIN      21
KARL        18
ALEX        17
HERMINE     12
ELEVEN      12
TWELVE      10
COLIN        6
Name: count, dtype: int64

In [41]:
hurdat.query('year==2022 & name == "TWELVE"')

,Unnamed: 0,date,time,record_id,status,latitude,longitude,max_wind,min_pressure,ne34ktr,...,se50ktr,sw50ktr,nw50ktr,ne64ktr,se64ktr,sw64ktr,nw64ktr,rad_max_wind,name,year
53856,55803,2022-10-04,1200,NaN,TD,13.6N,29.5W,30.0,1007.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0,TWELVE,2022
53857,55804,2022-10-04,1800,NaN,TD,14.2N,30.2W,30.0,1007.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0,TWELVE,2022
53858,55805,2022-10-05,0,NaN,TD,14.6N,30.5W,30.0,1007.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,TWELVE,2022
53859,55806,2022-10-05,600,NaN,TD,15.0N,30.9W,30.0,1007.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,TWELVE,2022
53860,55807,2022-10-05,1200,NaN,TD,15.3N,31.5W,30.0,1007.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,TWELVE,2022
53861,55808,2022-10-05,1800,NaN,TD,16.0N,31.9W,30.0,1007.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,TWELVE,2022
53862,55809,2022-10-06,0,NaN,TD,17.0N,32.1W,30.0,1007.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,TWELVE,2022
53863,55810,2022-10-06,600,NaN,TD,17.7N,32.8W,30.0,1007.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,TWELVE,2022
53864,55811,2022-10-06,1200,NaN,TD,18.1N,33.9W,30.0,1008.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,TWELVE,2022
53865,55812,2022-10-06,1800,NaN,TD,18.7N,35.0W,30.0,1009.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,TWELVE,2022
